In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt

import pymysql
import mysql.connector
from sqlalchemy import create_engine

pd.options.mode.chained_assignment = None

In [2]:
user = 'flatiron'
pwd = 'flatiron'
host = '127.0.0.1'
shporty = '3306'
db = 'stock'
connector = 'mysql+mysqlconnector://'+user+':'+pwd+'@'+host+':'+shporty+'/'+db
engine = create_engine(connector, echo=False)

In [10]:
list_path = '/home/steve/documents/flatIron/fIProject/Stocks/master_list_no_empty.csv'
csv_dir_path = '/home/steve/documents/finance/dbs/csvs/'

In [81]:
stock_list = pd.read_csv(list_path)
new_stock_list = stock_list.iloc[1691:]

In [42]:
# str -> df
# returns a dataframe of the given stock name
def read_stock_csv(name):
    # import format:
    # pd.read_csv(csv_dir_path + STOCK + '.csv')
    return pd.read_csv(csv_dir_path + name + '.csv')

In [43]:
# pd.DataFrame, String -> pd.DataFrame
# returns a cleaned dataframe to be added into the db
def clean_df(df, ticker):
    # drops
    to_drop = ['Dividends', 'Stock Splits', 'Adj Close']
    df.drop(columns=to_drop, inplace=True, errors='ignore', axis=0)
    df.dropna(inplace=True)
    
    # adds
    df['ticker'] = ticker
    
    return df

In [84]:
# Populates stockHist MySQL table with historical stock data
for ticker in new_stock_list['ticker']:
    try:
        stock_df = read_stock_csv(ticker)
        stock_df = clean_df(stock_df, ticker)
        stock_df.to_sql(name='stockHist', con=engine, if_exists='append', index=False)
    except:
        pass